# Custom functions for the loss, metrics and callbacks

Keras is highly customizable because most details of the API can be adapted to your needs. The following workflow presents the implementation of a custom loss function, custom metrics and custom training callbacks.

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
np.random.seed(1234)
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers.core import Dense

/home/stefan/py2_virtualenv/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Generate data

As simple example, data is taken from 2D-Gaussian distributions.

In [2]:
signal_mean = [1.0, 1.0]
signal_cov = [[1.0, 0.0],
              [0.0, 1.0]]
background_mean = [-1.0, -1.0]
background_cov = [[1.0, 0.0],
                  [0.0, 1.0]]
num_events = 100000

signal = np.random.multivariate_normal(signal_mean, signal_cov, num_events/2)
background = np.random.multivariate_normal(background_mean, background_cov, num_events/2)

inputs = np.vstack([signal, background])
labels = np.vstack([np.ones((num_events/2, 1)), np.zeros((num_events/2, 1))])

np.random.seed(1234)
np.random.shuffle(inputs)
np.random.seed(1234)
np.random.shuffle(labels)

## Define model

In [3]:
model = Sequential()
model.add(Dense(100, activation="relu", input_shape=(2,)))
model.add(Dense(1, activation="sigmoid"))

## Add custom loss function and custom metric

Custom loss functions and metrics are very similar because they just have to be a python callable satisfying the signature `function(y_true, y_pred)`.

In [4]:
import keras.backend as K

def custom_loss(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

def custom_metric(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

model.compile(loss=custom_loss, optimizer="adam", metrics=[custom_metric])

## Add custom training callback

Callbacks can be inherited from the base callback. To adapt the functionality, simply modify the methods of the class shown below performing actions at different steps of the training process.

The implemented callback calculates the true-positive and false-positive rate of the classifier after each epoch.

In [5]:
from keras.callbacks import Callback

class CustomCallback(Callback):
    def on_train_begin(self, logs={}):
        self.false_positive_rates = []
        self.true_positive_rates = []

    def on_train_end(self, logs={}):
        pass

    def on_epoch_begin(self, epoch, logs={}):
        pass

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.validation_data[0])
        y_true = self.validation_data[1]
        fp = 0.0
        tp = 0.0
        for t, p in zip(y_true, y_pred):
            if t == 0 and p > 0.5:
                fp += 1.0
            if t == 1 and p > 0.5:
                tp += 1.0
        self.false_positive_rates.append(fp/len(y_true))
        self.true_positive_rates.append(tp/len(y_true))

    def on_batch_begin(self, batch, logs={}):
        pass

    def on_batch_end(self, batch, logs={}):
        pass

custom_callback = CustomCallback()
model.fit(inputs, labels, batch_size=len(labels)/2, epochs=10, validation_split=0.5, callbacks=[custom_callback]);

Train on 50000 samples, validate on 50000 samples
Epoch 1/10
50000/50000 [==============================] - 0s 2us/step - loss: 0.2385 - custom_metric: 0.2385 - val_loss: 0.2332 - val_custom_metric: 0.2332
Epoch 2/10
50000/50000 [==============================] - 0s 2us/step - loss: 0.2328 - custom_metric: 0.2328 - val_loss: 0.2276 - val_custom_metric: 0.2276
Epoch 3/10
50000/50000 [==============================] - 0s 2us/step - loss: 0.2273 - custom_metric: 0.2273 - val_loss: 0.2222 - val_custom_metric: 0.2222
Epoch 4/10
50000/50000 [==============================] - 0s 1us/step - loss: 0.2218 - custom_metric: 0.2218 - val_loss: 0.2168 - val_custom_metric: 0.2168
Epoch 5/10
50000/50000 [==============================] - 0s 1us/step - loss: 0.2165 - custom_metric: 0.2165 - val_loss: 0.2116 - val_custom_metric: 0.2116
Epoch 6/10
50000/50000 [==============================] - 0s 1us/step - loss: 0.2112 - custom_metric: 0.2112 - val_loss: 0.2065 - val_custom_metric: 0.2065
Epoch 7/10
500

In [6]:
for i, (tpr, fpr) in enumerate(zip(custom_callback.true_positive_rates, custom_callback.false_positive_rates)):
    print("Epoch: {:>2}, TPR: {:.2f}, FPR: {:.2f}".format(i+1, tpr, fpr))

Epoch:  1, TPR: 0.20, FPR: 0.04
Epoch:  2, TPR: 0.24, FPR: 0.04
Epoch:  3, TPR: 0.28, FPR: 0.04
Epoch:  4, TPR: 0.32, FPR: 0.04
Epoch:  5, TPR: 0.39, FPR: 0.04
Epoch:  6, TPR: 0.44, FPR: 0.04
Epoch:  7, TPR: 0.45, FPR: 0.05
Epoch:  8, TPR: 0.46, FPR: 0.05
Epoch:  9, TPR: 0.46, FPR: 0.05
Epoch: 10, TPR: 0.47, FPR: 0.05
